# Agno

Agno is a lightweight library for building Multimodal Agents with memory, knowledge and tools.

docs link: https://docs.agno.com/

github link: https://github.com/agno-agi/agno

## Web Search Agent

In [1]:
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools.duckduckgo import DuckDuckGoTools
from dotenv import load_dotenv
load_dotenv()
import os

import nest_asyncio
nest_asyncio.apply()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

agent = Agent(
    model=Gemini(id="gemini-2.0-flash", api_key=GOOGLE_API_KEY, temperature=0.5),
    tools=[DuckDuckGoTools()],
    markdown=True
)
agent.print_response("What's happening in Dhaka?", stream=True)

Output()

In [3]:
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools.duckduckgo import DuckDuckGoTools

agent = Agent(
    model=Gemini(id="gemini-2.0-flash", api_key=GOOGLE_API_KEY, temperature=0.5),
    description="You are an enthusiastic news reporter with a flair for storytelling!",
    tools=[DuckDuckGoTools()],
    show_tool_calls=True,
    markdown=True
)
agent.print_response("Tell me about a breaking news story from Dhaka.", stream=True)

Output()

In [12]:
from agno.agent import Agent
from agno.models.google import Gemini
from agno.embedder.google import GeminiEmbedder
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
from agno.vectordb.lancedb import LanceDb, SearchType

agent = Agent(
    model=Gemini(id="gemini-2.0-flash", api_key=GOOGLE_API_KEY, temperature=0.5),
    description="You are a Thai cuisine expert!",
    instructions=[
        "Search your knowledge base for Bangladeshi recipes.",
        "If the question is better suited for the web, search the web to fill in gaps.",
        "Prefer the information in your knowledge base over the web results."
    ],
    knowledge=PDFUrlKnowledgeBase(
        urls=["https://banglarecipes.com.au/wp-content/uploads/2012/09/rownaks-bangladeshi-recipes-ebook.pdf"],
        vector_db=LanceDb(
            uri="tmp/lancedb",
            table_name="recipes",
            search_type=SearchType.hybrid,
            embedder=GeminiEmbedder(dimensions=1536),
        ),
    ),
    tools=[DuckDuckGoTools()],
    show_tool_calls=True,
    markdown=True
)

# Comment out after the knowledge base is loaded
if agent.knowledge is not None:
    agent.knowledge.load()

agent.print_response("How do I make Haleem?", stream=True)
agent.print_response("What is the history of Bangladeshi curry?", stream=True)

INFO     Loading knowledge base

INFO     Reading: https://banglarecipes.com.au/wp-content/uploads/2012/09/rownaks-bangladeshi-recipes-ebook.pdf

INFO     Inserting 47 documents

INFO     Added 47 documents to knowledge base

Output()

Output()

In [21]:
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.yfinance import YFinanceTools

web_agent = Agent(
    name="Web Agent",
    role="Search the web for information",
    model=Gemini(id="gemini-2.0-flash", api_key=GOOGLE_API_KEY, temperature=0.5),
    tools=[DuckDuckGoTools()],
    instructions="Always include sources",
    show_tool_calls=True,
    markdown=True,
)

finance_agent = Agent(
    name="Finance Agent",
    role="Get financial data",
    model=Gemini(id="gemini-2.0-flash", api_key=GOOGLE_API_KEY, temperature=0.5),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True)],
    instructions="Use tables to display data",
    show_tool_calls=True,
    markdown=True,
)

agent_team = Agent(
    team=[web_agent, finance_agent],
    model=Gemini(id="gemini-2.0-flash", api_key=GOOGLE_API_KEY),
    instructions=["Always include sources", "Use tables to display data"],
    show_tool_calls=True,
    markdown=True,
)

agent_team.print_response("What's the market outlook and financial performance of AI semiconductor companies?", stream=True)

DEBUG    Function: duckduckgo_search registered with duckduckgo

DEBUG    Function: duckduckgo_news registered with duckduckgo

DEBUG    Function: get_current_stock_price registered with yfinance_tools

DEBUG    Function: get_company_info registered with yfinance_tools

DEBUG    Function: get_analyst_recommendations registered with yfinance_tools

Output()

WARNING  Could not run function duckduckgo_search(max_results=5, query=AI semiconductor market trends drivers      
         challenges)

ERROR    https://lite.duckduckgo.com/lite/ 202 Ratelimit                                                           
         Traceback (most recent call last):                                                                        
           File "/home/z4hid/anaconda3/envs/brainai/lib/python3.11/site-packages/agno/tools/function.py", line 363,
         in execute                                                                                                
             self.result = self.function.entrypoint(**entrypoint_args, **self.arguments)                           
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                           
           File                                                                                                    
         "/home/z4hid/anaconda3/envs/brainai/lib/python3.11/site-packages/pydantic/_internal/_validate_call.py",   
         line 38, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File                                                                                                    
         "/home/z4hid/anaconda3/envs/brainai/lib/python3.11/site-packages/pydantic/_internal/_validate_call.py",   
         line 111, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "/home/z4hid/anaconda3/envs/brainai/lib/python3.11/site-packages/agno/tools/duckduckgo.py", line   
         68, in duckduckgo_search                                                                                  
             return json.dumps(ddgs.text(keywords=query, max_results=(self.fixed_max_results or max_results)),     
         indent=2)                                                                                                 
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^      
           File                                                                                                    
         "/home/z4hid/anaconda3/envs/brainai/lib/python3.11/site-packages/duckduckgo_search/duckduckgo_search.py", 
         line 292, in text                                                                                         
             raise DuckDuckGoSearchException(err)                                                                  
         duckduckgo_search.exceptions.DuckDuckGoSearchException: https://lite.duckduckgo.com/lite/ 202 Ratelimit

In [20]:
from agno.agent import Agent

agent = Agent(model=Gemini(id="gemini-2.0-flash-thinking-exp-01-21", api_key=GOOGLE_API_KEY),
              markdown=True, debug_mode=True)
agent.print_response("Share a 2 sentence horror story")

DEBUG    *********** Agent ID: 07024715-a3b5-4b0c-a2f0-b22dbabdfb5d ***********

DEBUG    *********** Session ID: 61f9bffd-93dd-405f-828b-99ed3b15d3ea ***********

DEBUG    *********** Agent Run Start: ad3961e2-fba9-4415-a330-9cd9e3220d2a ***********

DEBUG    ---------- Google Response Start ----------

DEBUG    ---------- Model: gemini-2.0-flash-thinking-exp-01-21 ----------

...

DEBUG    ---------- Google Response End ----------

DEBUG    Added 2 Messages to AgentMemory

DEBUG    Added AgentRun to AgentMemory

DEBUG    --**-- Logging Agent Run

DEBUG    *********** Agent Run End: ad3961e2-fba9-4415-a330-9cd9e3220d2a ***********